In [224]:
import numpy as np

In [225]:
from sklearn.datasets import load_iris

In [226]:
X = load_iris()["data"]
y = load_iris()["target"]

In [227]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [228]:
def relu(x):
    return np.maximum(0,x)

In [229]:
# def softmax(x):
#     c = np.max(x)
#     x = x-c
#     return np.exp(x)/np.sum(np.exp(x))

In [230]:
def softmax(x):
    c = np.max(x,axis=1).reshape(-1,1)
    x = x-c
    return np.exp(x)/np.sum(np.exp(x),axis=1).reshape(-1,1)

In [231]:
def categorical_crossentropy(t,y):
    return np.mean(-t*np.log(y))

In [232]:
def make_one(x):
    result = np.zeros((x.size, np.unique(x).size))
    for idx1,idx2 in enumerate(x):
        result[idx1,idx2] = 1
    return result

In [233]:
X.shape

(150, 4)

In [234]:
y = make_one(y)

In [235]:
input_shape = X.shape[1:]
output_shape = y.shape[1:]

In [236]:
W1 = np.random.randn(4,50)
b1 = np.zeros(50)
W2 = np.random.randn(50,3)
b2 = np.zeros(3)

In [237]:
def predict(x):
    W1 = np.random.randn(4,50)
    b1 = np.zeros(50)
    W2 = np.random.randn(50,3)
    b2 = np.zeros(3)
    layer1 = np.dot(x,W1) + b1
    z1 = sigmoid(layer1)
    layer2 = np.dot(z1,W2) + b2
    out = softmax(layer2)
    return out

In [238]:
categorical_crossentropy(y,y_hat)

1.3445492253165467

In [239]:
def numerical_gradient(f,x):
    h = 1e-4
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=["multi_index"],op_flags=["readwrite"])
    
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh = f(x)
        x[idx] = tmp_val
        fx = f(x)
        grad[idx] = (fxh-fx)/h
        it.iternext()
    return grad

## ex)

In [240]:
a = {}
a["w"] = np.arange(10)
a

{'w': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])}

In [241]:
a = {}
a["w1"] = "a"
len(a)

1

# Class 생성

In [242]:
class Network:
    def __init__(self):
        self.layer = {}
    
    def add(self,x1,x2,activation):
        activation_dict = {
            "sigmoid" : sigmoid,
            "relu" : relu,
            "softmax" : softmax
        }
        w = "W"+str(int(len(self.layer)/3+1))
        b = "b"+str(int(len(self.layer)/3+1))
        a = "activation"+str(int(len(self.layer)/3+1))
        self.layer[w] = np.random.randn(x1,x2)
        self.layer[b] = np.zeros(x2)
        self.layer[a] = activation_dict[activation]
        
    def predict(self,x):
        y = np.dot(x,self.W1) + self.b1
        y = relu(y)
        y = np.dot(y,self.W2) + self.b2
        y = softmax(y)
        return y
    
    def loss(self,x,t):
        y = self.predict(x)
        err = categorical_crossentropy(t,y) 
        self.err = err
        return self.err
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        t = np.argmax(t,axis=1)
        self.acc = np.sum(y==t)/t.size
        return self.acc
    
    def gradient(self,x,t):
        #self.loss(x,t)
        lr = 1e-4
        W_loss = lambda W : self.loss(x,t)
        dW1 = numerical_gradient(W_loss,self.W1)
        db1 = numerical_gradient(W_loss,self.b1)
        dW2 = numerical_gradient(W_loss,self.W2)
        db2 = numerical_gradient(W_loss,self.b2)
        self.W1 -= dW1*lr
        self.b1 -= db1*lr
        self.W2 -= dW2*lr
        self.b2 -= db2*lr
        
    def fit(self,x,t,epochs):
        for epoch in range(epochs):
            self.gradient(x,t)
            if epoch % 100 == 0:
                print(f'loss : {self.err} === accuracy : {self.accuracy(x,t)}')

In [243]:
model = Network()

In [244]:
model.add(4,50,"sigmoid")

In [245]:
model.add(50,3,"softmax")

In [246]:
model.layer["activation1"](10)

0.9999546021312976

In [247]:
# for _,v in model.layer.items():
#     print(v)
# for k,v in model.layer.key():
#     print(k)

In [248]:
# model.fit(X,y,10000)

In [249]:
model.predict(X[0:1])

AttributeError: 'Network' object has no attribute 'W1'

In [ ]:
np.argmax(model.predict(X[0:1]))

1

In [ ]:
model.loss(X,y)

13.67638397404389

In [ ]:
model.gradient(X,y)

In [ ]:
epochs = 10000
for epoch in range(epochs): 
    model.gradient(X,y)
    if epoch % 100 == 0:
        print(f'loss : {model.err} === accuracy : {model.accuracy(X,y)}')

loss : 13.665600786821658 === accuracy : 0.3333333333333333
loss : 12.770010759363242 === accuracy : 0.3333333333333333
loss : 11.98516958054729 === accuracy : 0.3333333333333333
loss : 11.214640863054386 === accuracy : 0.3333333333333333
loss : 10.450813360515575 === accuracy : 0.3333333333333333
loss : 9.695745851150892 === accuracy : 0.3333333333333333
loss : 8.965239931675825 === accuracy : 0.3466666666666667
loss : 8.407512376184082 === accuracy : 0.4666666666666667
loss : 8.039188808330472 === accuracy : 0.5266666666666666
loss : 7.718532977884472 === accuracy : 0.5533333333333333
loss : 7.411097051776952 === accuracy : 0.5733333333333334
loss : 7.110147177182449 === accuracy : 0.5933333333333334
loss : 6.814328375812549 === accuracy : 0.5933333333333334
loss : 6.523041670108632 === accuracy : 0.6
loss : 6.234782740253026 === accuracy : 0.6
loss : 5.948469106215778 === accuracy : 0.6
loss : 5.663499709844925 === accuracy : 0.6
loss : 5.380784028907326 === accuracy : 0.6
loss : 5.

KeyboardInterrupt: 

In [ ]:
X[[8]]

array([[4.4, 2.9, 1.4, 0.2]])

In [ ]:
y[8]

array([1., 0., 0.])

In [ ]:
np.argmax(model.predict(X[[8]]))

0

In [ ]:
np.argmax(y[8])

0